### Libraries

In [1]:
# pip install catboost

In [2]:
import numpy as np
import pandas as pd
import warnings

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [3]:
warnings.filterwarnings("ignore")

# 한글 처리
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

# 폰트 이름 얻어오기
font_name = fm.FontProperties(fname=font_path).get_name()

# 폰트 설정
plt.rcParams['font.family'] = font_name


### Load data

In [4]:
from dotenv import load_dotenv
import os
from utils.EDA import analysis_임신성공여부, vis_numeric_corr_matrix
load_dotenv()
df_train = pd.read_csv(os.getenv('TRAIN_DATA_PATH')).drop(columns=['ID'])
df_test = pd.read_csv(os.getenv('TEST_DATA_PATH')).drop(columns=['ID'])

df_train.head(8).to_csv('train_sample.csv', encoding = 'utf-8-sig', index = False)

# df_train = pd.read_csv('data/train.csv').drop(columns=['ID'])
# df_test = pd.read_csv('data/test.csv').drop(columns=['ID'])

### Preprocessing

#### 컬럼 제거

In [5]:
# 컬럼 제거
dropped_columns = ['불임 원인 - 여성 요인']

df_train = df_train.drop(columns=dropped_columns, axis=1)
df_test = df_test.drop(columns=dropped_columns, axis=1)

print("제거된 컬럼 개수:", len(dropped_columns))
print("제거된 컬럼:", dropped_columns)
print("df_train.shape:", df_train.shape)
print("df_test.shape:", df_test.shape)

제거된 컬럼 개수: 1
제거된 컬럼: ['불임 원인 - 여성 요인']
df_train.shape: (256351, 67)
df_test.shape: (90067, 66)


#### astype(int)

In [6]:
# '횟수'를 포함하는 컬럼 찾기
count_columns = [col for col in df_train.columns if '횟수' in col]

def extract_number(value):
    if isinstance(value, str):
        return int(value[0])  # 맨 앞자리 숫자로 변환
    return value

for col in count_columns:
    df_train[col] = df_train[col].apply(extract_number).astype(int)
    df_test[col] = df_test[col].apply(extract_number).astype(int)
print("변환된 컬럼:", count_columns)

변환된 컬럼: ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']


#### 특정 시술 유형 - ICSI / 배아 생성 주요 이유 - 현재 시술용

In [7]:
def get_dummies(df):
    # 특정 시술 유형
    df['특정 시술 유형 - ICSI'] = np.where(df['시술 유형'] != 'IVF', np.nan,  # 시술 유형이 DI
                                     np.where(df['특정 시술 유형'].str.contains('ICSI', na=False), 1, 0))
    df['특정 시술 유형 - Unknown'] = np.where(df['시술 유형'] != 'IVF', np.nan,  # 시술 유형이 DI
                                        np.where(df['특정 시술 유형'].str.contains('Unknown', na=False), 1, 0))

    # 배아 생성 주요 이유
    df['배아 이유 - 현재 시술용'] = np.where(df['시술 유형'] != 'IVF', np.nan,  # 시술 유형이 DI
                                    np.where(df['배아 생성 주요 이유'].str.contains('현재 시술용', na=False), 1, 0))
    df['배아 이유 - 배아 저장용'] = np.where(df['시술 유형'] != 'IVF', np.nan,  # 시술 유형이 DI
                                    np.where(df['배아 생성 주요 이유'].str.contains('배아 저장용', na=False), 1, 0))
    df['배아 이유 - 기증용'] = np.where(df['시술 유형'] != 'IVF', np.nan,  # 시술 유형이 DI
                                 np.where(df['배아 생성 주요 이유'].str.contains('기증용', na=False), 1, 0))
    
    df = df.drop(columns=['특정 시술 유형', '배아 생성 주요 이유'])
    
    return df

df_train = get_dummies(df_train)
df_test = get_dummies(df_test)

#### 남성/여성/부부 주/부 불임 원인 제거

In [8]:
# 남성, 여성, 부부의 불임 원인 통합 (주, 부)
def infertility_cause(df):
    df['남성 불임 원인'] = ((df['남성 주 불임 원인'] == 1) | (df['남성 부 불임 원인'] == 1)).astype(int)
    df['여성 불임 원인'] = ((df['여성 주 불임 원인'] == 1) | (df['여성 부 불임 원인'] == 1)).astype(int)
    df['부부 불임 원인'] = ((df['부부 주 불임 원인'] == 1) | (df['부부 부 불임 원인'] == 1)).astype(int)
    
    drop_cols = ['남성 주 불임 원인', '남성 부 불임 원인',
                 '여성 주 불임 원인', '여성 부 불임 원인',
                 '부부 주 불임 원인', '부부 부 불임 원인']
    
    df = df.drop(columns=drop_cols, errors='ignore')

    return df

df_train = infertility_cause(df_train)
df_test = infertility_cause(df_test)

#### 배아 해동 경과일이 0보다 큰 값을 가지는 경우 동결 배아 사용 여부 1로 바꾸기

In [9]:
def process(df):
    df.loc[(df['배아 해동 경과일'] > 0), '동결 배아 사용 여부'] = 1
    # df.loc[(df['미세주입된 난자 수'] > 0), '특정 시술 유형 - ICSI'] = 1
    return df

df_train = process(df_train)
df_test = process(df_test)

### Feature engineering

In [10]:
def feature_engineering(df):
   
    # 임신/출산 성공률
    df['IVF 시술 대비 임신 성공률'] = df['IVF 임신 횟수'] / df['IVF 시술 횟수']
    
    # 난자
    df['총 난자 수'] = df['수집된 신선 난자 수'] + df['해동 난자 수']
    df['난자 사용률'] = df['혼합된 난자 수'] / df['총 난자 수']
    
    # 배아
    df['총 배아 수'] = df['총 생성 배아 수'] + df['해동된 배아 수']
    df['미세주입 배아 생성 확률'] = df['미세주입에서 생성된 배아 수'] / df['미세주입된 난자 수']
    df['미세주입 배아 이식 확률'] = df['미세주입 배아 이식 수'] / df['미세주입에서 생성된 배아 수']
    df['총 배아 생성 확률'] = df['총 배아 수'] / df['총 난자 수']
    df['신선 + 기증 + 동결'] = df[['신선 배아 사용 여부', '기증 배아 사용 여부', '동결 배아 사용 여부']].sum(axis=1)
    df['배아 저장 비율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1e-5)
    
    # 이식된 배아 수 관련
    df['배아 이식 대비 임신 성공률'] = df['총 임신 횟수'] / df['이식된 배아 수']
    # df['배아 이식 대비 임신 성공률'] = df['배아 이식 대비 임신 성공률'].apply(lambda x: 1 if (x > 1 and x != np.inf) else x)
    df['배아 이식 대비 출산 성공률'] = df['총 출산 횟수'] / df['이식된 배아 수']
    # df['배아 이식 대비 출산 성공률'] = df['배아 이식 대비 출산 성공률'].apply(lambda x: 1 if (x > 1 and x != np.inf) else x)
    df['배아 이식 확률'] = df['이식된 배아 수'] / df['총 배아 수']
    df['이식된 배아 대비 이식 기간'] = df['배아 이식 경과일'] / df['이식된 배아 수']

    # 기타
    df['배란자극 * 단일이식'] = df['배란 자극 여부'] * df['단일 배아 이식 여부']
    df['배란자극 * ICSI'] = df['배란 자극 여부'] * df['특정 시술 유형 - ICSI'] 
    df['경과일 차이 - 난자 혼합 * 배아 이식'] = df['배아 이식 경과일'] - df['난자 혼합 경과일']
    df['경과일 차이 - 배아 해동 * 배아 이식'] = df['배아 이식 경과일'] - df['배아 해동 경과일']
    df['경과일 차이 - 난자 혼합 * 배아 해동'] = df['배아 해동 경과일'] - df['난자 혼합 경과일']
    
    # 바이너리 합
    binary_cols = ['배란 자극 여부', '단일 배아 이식 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부']
    df['바이너리 합'] = df[binary_cols].sum(axis=1)

    # PGD, PGS 합
    pgd_pgs = ['착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', 'PGD 시술 여부', 'PGS 시술 여부']
    df['PGD * PGS 합'] = df[pgd_pgs].sum(axis=1)
    
    # 불임 원인 점수
    infertility_cols = ['불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', 
                        '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
                        '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태']
    for col in infertility_cols:
        if df[col].dtype == 'object':
            df[col] = df[col].map({'Y': 1, 'N': 0})
    df['불임원인 수'] = df[infertility_cols].sum(axis=1)

    # 변수 제거
    cols = ['총 난자 수', '난자 해동 경과일']
    df = df.drop(columns=cols, errors='ignore')

    return df

df_train = feature_engineering(df_train)
df_test = feature_engineering(df_test)

#### 환자 & 난자 기증자 나이 비교

In [11]:
def compare_age(df):
    patient_age_map = {'만18-34세': 4, '만35-37세': 5, '만38-39세': 5, '만40-42세': 6, '만43-44세': 6, '만45-50세': 7, '알 수 없음': np.nan}
    egg_age_map = {'만20세 이하': 1, '만21-25세': 2, '만26-30세': 3, '만31-35세': 4, '만36-40세': 5, '만41-45세': 6, '알 수 없음': np.nan}
    sper_age_map = {'만20세 이하': 1, '만21-25세': 2, '만26-30세': 3, '만31-35세': 4, '만36-40세': 5, '만41-45세': 6, '알 수 없음': np.nan}
    
    df['시술 당시 나이 순위'] = df['시술 당시 나이'].map(patient_age_map)
    df['난자 기증자 나이 순위'] = df['난자 기증자 나이'].map(egg_age_map)
    df['정자 기증자 나이 순위'] = df['정자 기증자 나이'].map(sper_age_map)

    df['여성 나이 순위'] = np.where(
        df['난자 출처'] == '기증 제공', df['난자 기증자 나이 순위'],
        np.where(df['난자 출처'] == '본인 제공', df['시술 당시 나이 순위'], np.nan)
    )

    df['여성 나이 비교'] = df.apply(
        lambda row: np.nan if pd.isna(row['시술 당시 나이 순위']) or pd.isna(row['난자 기증자 나이 순위']) 
        else (2 if row['시술 당시 나이 순위'] > row['난자 기증자 나이 순위']
        else (1 if row['시술 당시 나이 순위'] == row['난자 기증자 나이 순위']
        else 0)), 
        axis=1
    )
    df.loc[df['난자 출처'] == '본인 제공', '여성 나이 비교'] = df['여성 나이 비교'].fillna(1)

    df['남성 나이 비교'] = df.apply(
        lambda row: np.nan if pd.isna(row['여성 나이 순위']) or pd.isna(row['정자 기증자 나이 순위']) 
        else (2 if row['여성 나이 순위'] > row['정자 기증자 나이 순위']
        else (1 if row['여성 나이 순위'] == row['정자 기증자 나이 순위'] 
        else 0)), 
        axis=1
    )

    df = df.drop(columns=['시술 당시 나이 순위', '난자 기증자 나이 순위', '정자 기증자 나이 순위', '여성 나이 순위'], errors='ignore')

    return df

df_train = compare_age(df_train)
df_test = compare_age(df_test)

#### 시술 유형이 DI인 경우 -1로 채우기

In [12]:
def di_null(df):
    mask = df['시술 유형'] == 'DI'
    columns_to_fill = [col for col in df.columns if col != '임신 시도 또는 마지막 임신 경과 연수']
    df.loc[mask, columns_to_fill] = df.loc[mask, columns_to_fill].fillna(-1)
    return df

df_train = di_null(df_train)
df_test = di_null(df_test)

In [13]:
# 결측 비율
missing_ratio = df_train.isnull().mean() * 100
print(missing_ratio)

시술 시기 코드                  0.000000
시술 당시 나이                  0.000000
임신 시도 또는 마지막 임신 경과 연수    96.344855
시술 유형                     0.000000
배란 자극 여부                  0.000000
                           ...    
바이너리 합                    0.000000
PGD * PGS 합               0.000000
불임원인 수                    0.000000
여성 나이 비교                  0.701772
남성 나이 비교                 90.083908
Length: 88, dtype: float64


#### NaN, inf 처리

In [14]:
# 결측값 대체
def replace_inf_and_nan(df, value_inf, value_na):
    df.replace([np.inf, -np.inf], value_inf, inplace=True)  # inf, -inf 변환
    df.fillna(value_na, inplace=True)  # NaN 변환
    return df

# 변환 적용
value_inf = 999 # 3 / 0
value_na = 999 # 0 / 0
df_train = replace_inf_and_nan(df_train, value_inf, value_na)
df_test = replace_inf_and_nan(df_test, value_inf, value_na)

#### Encoding

In [15]:
# object 타입 컬럼 확인
cat_features = list(df_train.select_dtypes(include=['object']).columns)
cat_features

['시술 시기 코드',
 '시술 당시 나이',
 '시술 유형',
 '배란 유도 유형',
 '난자 출처',
 '정자 출처',
 '난자 기증자 나이',
 '정자 기증자 나이']

In [16]:
# LabelEncoder 적용
for col in cat_features:
    le = LabelEncoder() 
    df_train[col] = le.fit_transform(df_train[col])  
    df_test[col] = le.transform(df_test[col]) 

#### Scaling

In [17]:
# scaling_cols = [
#     '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', 
#     '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
#     '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수',
#     '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 혼합 경과일',
#     '배아 이식 경과일', '경과일_차이', '불임원인_수', '배아저장비율', '이식된 배아 대비 이식 기간',
#     '총 배아 수', '나이 그룹별 평균 생성 배아 수', '나이 그룹별 평균 이식 배아 수', '나이 그룹별 평균 배아 이식 경과일'
# ]

# scaler = MinMaxScaler()
# df_train[scaling_cols] = scaler.fit_transform(df_train[scaling_cols])
# df_test[scaling_cols] = scaler.transform(df_test[scaling_cols])

### Modeling

In [18]:
X = df_train.drop('임신 성공 여부', axis=1)
y = df_train['임신 성공 여부']
X_test = df_test

#### Stratified K-Fold 

In [19]:
# Stratified K-Fold 설정
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)

metrics = {model: [] for model in ['CatBoost']} # Ensemble
feature_importances = {model: [] for model in ['CatBoost']}
test_proba = {model: [] for model in ['CatBoost']} # Ensemble


for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"===== Fold {fold} =====")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # 모델 정의
    cat_model = CatBoostClassifier(
        iterations=745, learning_rate=0.038577, depth=8, l2_leaf_reg=9.587765,subsample=0.748324, random_strength=0.0,
        min_data_in_leaf=59, leaf_estimation_iterations=1, loss_function='Logloss', eval_metric='AUC', verbose=100, random_seed=123
    )


    # 모델 학습
    for model in [cat_model]: # ensemble_model
        model.fit(X_train, y_train)

    # 평가 함수
    def evaluate_model(model, X_val, y_true):
        y_pred = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_val)[:, 1]

        y_pred_proba = np.where(X_val['시술 당시 나이'] == 6, 0, y_pred_proba)
        # y_pred_proba = np.where(X_val['이식된 배아 수'] == 0, 0, y_pred_proba)

        return {
            'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred),
            'Recall': recall_score(y_true, y_pred),
            'F1 Score': f1_score(y_true, y_pred),
            'ROC AUC Score': roc_auc_score(y_true, y_pred_proba)
        }

    # 평가 및 변수 중요도 저장
    for model_name, model in zip(metrics.keys(), [cat_model]): # ensemble_model
        metrics[model_name].append(evaluate_model(model, X_val, y_val))

    for model_name, model in zip(['CatBoost'], [cat_model]):
        feature_importances[model_name].append(model.feature_importances_)

    y_test_proba = cat_model.predict_proba(X_test)[:, 1]
    y_test_proba = np.where(X_test['시술 당시 나이'] == 6, 0, y_test_proba)
    
    # y_test_proba = np.where(X_test['이식된 배아 수'] == 0, 0, y_test_proba)
    
    # 테스트 데이터 예측 확률 저장
    test_proba['CatBoost'].append(y_test_proba)

===== Fold 1 =====
0:	total: 72.4ms	remaining: 53.8s
100:	total: 1.09s	remaining: 6.98s
200:	total: 2.08s	remaining: 5.63s
300:	total: 2.88s	remaining: 4.25s
400:	total: 3.71s	remaining: 3.19s
500:	total: 4.5s	remaining: 2.19s
600:	total: 5.3s	remaining: 1.27s
700:	total: 6.12s	remaining: 384ms
744:	total: 6.47s	remaining: 0us
===== Fold 2 =====
0:	total: 12.8ms	remaining: 9.54s
100:	total: 1.09s	remaining: 6.92s
200:	total: 2.03s	remaining: 5.5s
300:	total: 2.92s	remaining: 4.32s
400:	total: 3.79s	remaining: 3.25s
500:	total: 4.59s	remaining: 2.23s
600:	total: 5.41s	remaining: 1.3s
700:	total: 6.25s	remaining: 393ms
744:	total: 6.62s	remaining: 0us
===== Fold 3 =====
0:	total: 12.9ms	remaining: 9.63s
100:	total: 1.04s	remaining: 6.64s
200:	total: 2.02s	remaining: 5.47s
300:	total: 2.91s	remaining: 4.29s
400:	total: 3.71s	remaining: 3.19s
500:	total: 4.57s	remaining: 2.23s
600:	total: 5.43s	remaining: 1.3s
700:	total: 6.28s	remaining: 394ms
744:	total: 6.64s	remaining: 0us
===== Fold 4

In [20]:
# Val ROC AUC 기준 가중치 적용
print("===== Val ROC AUC 기준 가중치 적용 =====")
print("===== Stratified K-Fold 평균 성능 =====")
for model_name, model_metrics in metrics.items():
    # 모든 fold의 ROC AUC 점수 합계 계산
    total_val_auc = sum(fold_metric['ROC AUC Score'] for fold_metric in model_metrics)
    
    # 각 fold의 가중치 계산
    fold_weights = [fold_metric['ROC AUC Score'] / total_val_auc for fold_metric in model_metrics]
    
    # 각 metric별 가중 평균 계산
    weighted_avg_metrics = {
        metric: sum((fold_metric['ROC AUC Score'] / total_val_auc) * fold_metric[metric]
                    for fold_metric in model_metrics)
        for metric in model_metrics[0]
    }
    
    print(f"\n== {model_name} Model ==")
    print("Fold weights:", fold_weights)
    for metric, value in weighted_avg_metrics.items():
        print(f"{metric}: {value:.6f}")

===== Val ROC AUC 기준 가중치 적용 =====
===== Stratified K-Fold 평균 성능 =====

== CatBoost Model ==
Fold weights: [0.1995958547639919, 0.19999727236737666, 0.1998985586600518, 0.20013871140137166, 0.20036960280720795]
Accuracy: 0.746203
Precision: 0.539453
Recall: 0.120522
F1 Score: 0.197012
ROC AUC Score: 0.740370


In [21]:
# Val ROC AUC 계산 후 Softmax 변환 (정규화) 
print("===== Val ROC AUC 계산 후 Softmax 변환 (정규화)  =====")
print("===== Stratified K-Fold 평균 성능 =====")
T = 0.05
for model_name, model_metrics in metrics.items():
    roc_auc_scores = np.array([fold_metric['ROC AUC Score'] for fold_metric in model_metrics])
    
    # Softmax 변환 적용: 지수 함수를 통해 가중치를 계산하고 정규화
    exp_scores = np.exp(roc_auc_scores / T)
    total_exp = np.sum(exp_scores)
    fold_weights = exp_scores / total_exp
    
    # 각 metric별 가중 평균 계산
    weighted_avg_metrics = {
        metric: sum(fw * fold_metric[metric] for fw, fold_metric in zip(fold_weights, model_metrics))
        for metric in model_metrics[0]
    }
    
    print(f"\n== {model_name} Model ==")
    print("Fold weights:", fold_weights.tolist())
    for metric, value in weighted_avg_metrics.items():
        print(f"{metric}: {value:.6f}")

===== Val ROC AUC 계산 후 Softmax 변환 (정규화)  =====
===== Stratified K-Fold 평균 성능 =====

== CatBoost Model ==
Fold weights: [0.1940692961768846, 0.19992354361233328, 0.19846773712450846, 0.20202808466239897, 0.20551133842387478]
Accuracy: 0.746198
Precision: 0.539406
Recall: 0.120511
F1 Score: 0.196993
ROC AUC Score: 0.740387


In [22]:
# 0.740387 -> 0.740374

In [23]:
# 최종 변수 중요도 평균 계산
df_fi_list = []
for model_name, fi_list in feature_importances.items():
    avg_importance = np.mean(fi_list, axis=0)
    df_fi = pd.DataFrame({
        'Feature': X_train.columns,
        model_name: avg_importance 
    })
    df_fi = df_fi.sort_values(by=model_name, ascending=False).reset_index(drop=True)
    df_fi_list.append(df_fi)
    
df_fi_final = pd.concat(df_fi_list, axis=1)
df_fi_final.round(6)

,Feature,CatBoost
0,이식된 배아 수,32.881075
1,배아 이식 대비 출산 성공률,16.348744
2,배아 이식 경과일,9.338451
3,시술 당시 나이,7.770922
4,배아 저장 비율,3.259943
...,...,...
82,신선 배아 사용 여부,0.000194
83,불임 원인 - 자궁경부 문제,0.000064
84,불임 원인 - 정자 면역학적 요인,0.000000
85,시술 유형,0.000000


In [24]:
abs(df_train.corr()['임신 성공 여부']).sort_values(ascending=False)

임신 성공 여부              1.000000
배아 이식 대비 임신 성공률       0.239867
배아 이식 대비 출산 성공률       0.239861
이식된 배아 대비 이식 기간       0.237093
배아 이식 경과일             0.237010
                        ...   
불명확 불임 원인             0.001473
여성 나이 비교              0.001203
불임 원인 - 정자 면역학적 요인    0.001166
불임 원인 - 정자 농도         0.000626
불임 원인 - 난관 질환         0.000544
Name: 임신 성공 여부, Length: 88, dtype: float64

In [25]:
# # catboost 기준 변수 중요도 0.05 미만 변수 제거
# df_cat = df_fi_final.iloc[:, :2]
# df_selected = df_cat[df_cat['CatBoost'] > 0.05]
# selected_features = df_selected['Feature'].tolist()

# # 중요도가 높은 피처만 선택하여 새로운 데이터 생성
# X = X[selected_features]
# X_test = X_test[selected_features]

### Re-modeling

In [26]:
# # Stratified K-Fold 설정
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)

# metrics = {model: [] for model in ['CatBoost']} # Ensemble
# feature_importances = {model: [] for model in ['CatBoost']}
# test_proba = {model: [] for model in ['CatBoost']} # Ensemble

# # metrics = {model: [] for model in ['CatBoost', 'XGBoost', 'LightGBM', 'AdaBoost']} # Ensemble
# # feature_importances = {model: [] for model in ['CatBoost', 'XGBoost', 'LightGBM', 'AdaBoost']}
# # test_proba = {model: [] for model in ['CatBoost', 'XGBoost', 'LightGBM', 'AdaBoost']} # Ensemble

# for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
#     print(f"===== Fold {fold} =====")

#     X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     # 모델 정의
#     cat_model = CatBoostClassifier(
#     iterations=745, 
#     learning_rate=0.038577, 
#     depth=8, 
#     l2_leaf_reg=9.587765,
#     subsample=0.748324, 
#     random_strength=0.0,  # 0으로 설정
#     min_data_in_leaf=59, 
#     leaf_estimation_iterations=1, 
#     loss_function='Logloss', 
#     eval_metric='AUC', 
#     verbose=100, 
#     random_seed=123)

#     # xgb_model = XGBClassifier(
#     #     n_estimators=700, learning_rate=0.03, max_depth=7, min_child_weight=3,
#     #     gamma=0.1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1,
#     #     reg_lambda=1.0, verbosity=1, random_state=123
#     # )

#     # lgbm_model = LGBMClassifier(
#     #     n_estimators=700, learning_rate=0.03, max_depth=-1, num_leaves=64,
#     #     min_child_samples=20, subsample=0.8, colsample_bytree=0.8,
#     #     reg_alpha=0.1, reg_lambda=1.0, verbosity=1, random_state=123
#     # )
    
#     # adaboost_model = AdaBoostClassifier(
#     #     estimator=DecisionTreeClassifier(max_depth=2, min_samples_split=10, min_samples_leaf=5, random_state=123),
#     #     n_estimators=500, learning_rate=0.05,
#     #     algorithm="SAMME", random_state=123
#     # )
    
# #     ensemble_model = VotingClassifier(
# #         estimators=[('catboost', cat_model), ('xgboost', xgb_model), ('lightgbm', lgbm_model), ('adaboost', adaboost_model)],
# #         voting='soft', weights=[1, 1, 1, 0.8]
# #     )


#     # 모델 학습
#     for model in [cat_model]: # ensemble_model
# #     for model in [cat_model, xgb_model, lgbm_model, adaboost_model]: # ensemble_model
#         model.fit(X_train, y_train)

#     # 평가 함수
#     def evaluate_model(model, X_val, y_true):
#         y_pred = model.predict(X_val)
#         y_pred_proba = model.predict_proba(X_val)[:, 1]
        
#         # X_val의 이식된 배아 수가 0이면 y_pred_proba가 0이 되게 함
#         y_pred_proba = np.where(X_val['이식된 배아 수'] == 0, 0, y_pred_proba)

#         return {
#             'Accuracy': accuracy_score(y_true, y_pred),
#             'Precision': precision_score(y_true, y_pred),
#             'Recall': recall_score(y_true, y_pred),
#             'F1 Score': f1_score(y_true, y_pred),
#             'ROC AUC Score': roc_auc_score(y_true, y_pred_proba)
#         }

#     # 평가 및 변수 중요도 저장
#     for model_name, model in zip(metrics.keys(), [cat_model]): # ensemble_model
#         metrics[model_name].append(evaluate_model(model, X_val, y_val))

#     for model_name, model in zip(['CatBoost'], [cat_model]):
#         feature_importances[model_name].append(model.feature_importances_)
        
#     # 테스트 데이터 예측 확률 저장
#     # [ADD] 이식된 배아 수가 0이면 0으로 예측하도록 함.
#     y_test_proba = cat_model.predict_proba(X_test)[:, 1]
#     y_test_proba = np.where(X_test['이식된 배아 수'] == 0, 0, y_test_proba)
    
#     test_proba['CatBoost'].append(y_test_proba)
# #     test_proba['XGBoost'].append(xgb_model.predict_proba(X_test)[:, 1])
# #     test_proba['LightGBM'].append(lgbm_model.predict_proba(X_test)[:, 1])
# #     test_proba['AdaBoost'].append(adaboost_model.predict_proba(X_test)[:, 1])
# #     test_proba['Ensemble'].append(ensemble_model.predict_proba(X_test)[:, 1])

In [27]:
# # 평가 지표 평균 출력
# print("===== Stratified K-Fold 평균 성능 =====")
# for model_name, model_metrics in metrics.items():
#     avg_metrics = {metric: np.mean([fold_metric[metric] for fold_metric in model_metrics]) for metric in model_metrics[0]}
    
#     print(f"\n== {model_name} Model ==")
#     for metric, value in avg_metrics.items():
#         print(f"{metric}: {value:.6f}")

### Prediction

In [28]:
# # Best AUC 기록한 모델의 pred_proba로 선택
# pred_proba = np.mean(test_proba['CatBoost'], axis=0)
# # pred_proba = np.mean(test_proba['XGBoost'], axis=0)
# # pred_proba = np.mean(test_proba['LightGBM'], axis=0)
# # pred_proba = np.mean(test_proba['AdaBoost'], axis=0)
# # pred_proba = np.mean(test_proba['Ensemble'], axis=0)

In [29]:
# Fold별 Val ROC AUC 기준 가중치 적용한 pred_proba
model = 'CatBoost'
roc_auc_scores = np.array([fold_metrics['ROC AUC Score'] for fold_metrics in metrics[model]])
weights = roc_auc_scores / np.sum(roc_auc_scores)  # 합이 1이 되도록 정규화
test_proba_array = np.array(test_proba[model])
pred_proba = np.average(test_proba_array, axis=0, weights=fold_weights) # fold_weights: 정규화된 가중치

### Submission

In [30]:
# sample_submission = pd.read_csv('data/sample_submission.csv')
# sample_submission.head()

sample_submission = pd.read_csv(os.getenv('SUBMISSION_DATA_PATH'))
sample_submission.head()

,ID,probability
0,TEST_00000,0.0
1,TEST_00001,0.0
2,TEST_00002,0.0
3,TEST_00003,0.0
4,TEST_00004,0.0


In [31]:
#%% 
index = X_train[X_train['이식된 배아 수'] == 0][['이식된 배아 수']].index
pred_proba[index].min(), pred_proba[index].max()

# index.shape


IndexError: index 90068 is out of bounds for axis 0 with size 90067

In [ ]:
#%% 
index = X_test[X_test['이식된 배아 수'] == 0][['이식된 배아 수']].index
pred_proba[index].min(), pred_proba[index].max()

# index.shape


(0.0, 0.042472807310189764)

In [ ]:
# sample_submission['probability'] = pred_proba
# # 저장
# sample_submission.to_csv('data/submission.csv', index=False)
# sample_submission.head()


sample_submission['probability'] = pred_proba
# 저장
import datetime 
now = datetime.datetime.now()
save_path = os.path.join(f'./log/submission/{now.strftime("%Y%m%d_%H%M%S")}_{weighted_avg_metrics["ROC AUC Score"]:.5f}_eiden.csv')
sample_submission.to_csv(save_path, index=False)
sample_submission.head()

NameError: name 'y_' is not defined

In [ ]:
# 확인용
submission = pd.read_csv(save_path)
submission.head()

,ID,probability
0,TEST_00000,0.002249
1,TEST_00001,0.004002
2,TEST_00002,0.144358
3,TEST_00003,0.114534
4,TEST_00004,0.500987
